In [55]:
from scripts import commons

input_file = 'data/autotagging_moodtheme.tsv'
tracks, tags, extra = commons.read_file(input_file)

Reading: 18486 tracks, 4506 albums, 1533 artists


In [56]:
tracks

{948: {'artist_id': 87,
  'album_id': 149,
  'path': '48/948.mp3',
  'duration': 212.7,
  'tags': ['mood/theme---background'],
  'genre': set(),
  'instrument': set(),
  'mood/theme': {'background'}},
 950: {'artist_id': 87,
  'album_id': 149,
  'path': '50/950.mp3',
  'duration': 248.0,
  'tags': ['mood/theme---background'],
  'genre': set(),
  'instrument': set(),
  'mood/theme': {'background'}},
 951: {'artist_id': 87,
  'album_id': 149,
  'path': '51/951.mp3',
  'duration': 199.7,
  'tags': ['mood/theme---background'],
  'genre': set(),
  'instrument': set(),
  'mood/theme': {'background'}},
 2165: {'artist_id': 326,
  'album_id': 347,
  'path': '65/2165.mp3',
  'duration': 229.0,
  'tags': ['mood/theme---film'],
  'genre': set(),
  'instrument': set(),
  'mood/theme': {'film'}},
 2263: {'artist_id': 320,
  'album_id': 366,
  'path': '63/2263.mp3',
  'duration': 494.7,
  'tags': ['mood/theme---melancholic'],
  'genre': set(),
  'instrument': set(),
  'mood/theme': {'melancholic'}},

In [57]:
tags

defaultdict(dict,
            {'mood/theme': {'background': {948,
               950,
               951,
               4632,
               4640,
               18235,
               103599,
               103663,
               159797,
               159801,
               159807,
               159815,
               159834,
               244744,
               244843,
               245328,
               245348,
               245375,
               245388,
               245406,
               245421,
               282626,
               387423,
               402591,
               506956,
               588700,
               668360,
               668362,
               668363,
               668364,
               668366,
               709422,
               709627,
               709628,
               709629,
               718297,
               718298,
               718299,
               718300,
               718302,
               718304,
               718305,
  

In [321]:
import numpy as np

song1 = np.load('path/npy/00/7400.npy')
print(np.shape(song1))
song2 = np.load('path/audio/00/7400.npy')
print(np.shape(song2))

(96, 6845)
(96, 6845)


In [322]:
(song1==song2)

array([[False,  True, False, ..., False, False,  True],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True, False, ...,  True, False,  True],
       ...,
       [False,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [11]:
np.mean(song1)

-21.6335

In [15]:
tag_list = np.load("/home/rai/다운로드/mtg-jamendo-dataset-master/tag_list.npy")
tag_list = tag_list[127:]
tag_list

array(['mood/theme---action', 'mood/theme---adventure',
       'mood/theme---advertising', 'mood/theme---background',
       'mood/theme---ballad', 'mood/theme---calm',
       'mood/theme---children', 'mood/theme---christmas',
       'mood/theme---commercial', 'mood/theme---cool',
       'mood/theme---corporate', 'mood/theme---dark', 'mood/theme---deep',
       'mood/theme---documentary', 'mood/theme---drama',
       'mood/theme---dramatic', 'mood/theme---dream',
       'mood/theme---emotional', 'mood/theme---energetic',
       'mood/theme---epic', 'mood/theme---fast', 'mood/theme---film',
       'mood/theme---fun', 'mood/theme---funny', 'mood/theme---game',
       'mood/theme---groovy', 'mood/theme---happy', 'mood/theme---heavy',
       'mood/theme---holiday', 'mood/theme---hopeful',
       'mood/theme---inspiring', 'mood/theme---love',
       'mood/theme---meditative', 'mood/theme---melancholic',
       'mood/theme---melodic', 'mood/theme---motivational',
       'mood/theme---movie',

In [1]:
import torch
import torch.nn as nn
from model import CNN
import numpy as np
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = CNN(56).to(device=device)
model_state_dict = torch.load('mood_models/best_model.pth')
model.load_state_dict(model_state_dict)
model.dense

Linear(in_features=64, out_features=56, bias=True)

In [998]:
batch_size = 8
npy_bat = np.zeros((batch_size,96,1400))
for n in range(1,batch_size + 1):
    file = '/home/rai/다운로드/emotifymusic/rock/mel/' + str(n) + '.npy'
    npy = np.load(file, allow_pickle=True)
    npy_bat[n - 1, :, :] = npy[:, 0:1400]
npy_bat = npy_bat.astype(np.float32)

In [999]:
npy = npy_bat
npy = torch.from_numpy(npy)

npy = npy.to(device)
print(npy.shape)
#Sigmoid

torch.Size([8, 96, 1400])


In [1116]:
result = model(npy)

In [1117]:
np.shape(result)

torch.Size([8, 56])

In [1118]:
result = result.cpu()
result = result.detach().numpy()

In [1119]:
np.max(result)

0.45557845

In [1120]:
result_tag = []
for k in range(batch_size):
    sigmoid = result[k]
    tag = tag_list[sigmoid>=0.3]
    
    if tag.size == 0:
        result_tag.append('nothing')
    else:
        result_tag.append(tag)
        
result_tag

['nothing',
 'nothing',
 'nothing',
 'nothing',
 array(['mood/theme---relaxing'], dtype='<U31'),
 'nothing',
 'nothing',
 'nothing']

In [1139]:
import csv
import pandas as pd

f = open("/home/rai/다운로드/mtg-jamendo-dataset-master/data/splits/split-0/autotagging_moodtheme-test.tsv", encoding='utf8')
reader = csv.reader(f, delimiter='\t')

mood = []
for i in reader:
    mood.append(i)
f.close()
mood = pd.DataFrame(mood)
mood

,0,1,2,3,4,5,6,7,8,9,10,11
0,TRACK_ID,ARTIST_ID,ALBUM_ID,PATH,DURATION,TAGS,None,None,None,None,None,None
1,track_0003524,artist_000530,album_000552,24/3524.mp3,266.1,mood/theme---melodic,None,None,None,None,None,None
2,track_0004882,artist_000696,album_000751,82/4882.mp3,210.4,mood/theme---meditative,None,None,None,None,None,None
3,track_0004883,artist_000696,album_000751,83/4883.mp3,249.5,mood/theme---meditative,None,None,None,None,None,None
4,track_0004884,artist_000696,album_000751,84/4884.mp3,220.7,mood/theme---meditative,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4227,track_1420586,artist_497202,album_165688,86/1420586.mp3,292.7,mood/theme---deep,mood/theme---summer,None,None,None,None,None
4228,track_1420587,artist_497202,album_165688,87/1420587.mp3,202.7,mood/theme---deep,mood/theme---summer,None,None,None,None,None
4229,track_1420588,artist_497202,album_165688,88/1420588.mp3,251.9,mood/theme---deep,mood/theme---summer,None,None,None,None,None
4230,track_1420849,artist_496074,album_165717,49/1420849.mp3,314.3,mood/theme---dramatic,None,None,None,None,None,None


In [95]:
import torch
import torch.nn as nn
from model import CNN
import numpy as np
device = torch.device('cuda:0')
model = CNN(56).to(device=device)
model_state_dict = torch.load('mood_models/best_model.pth')
model.load_state_dict(model_state_dict)
model.dense

Linear(in_features=64, out_features=56, bias=True)

In [96]:
batch_size = 8
npy_bat = np.zeros((batch_size,96,1400))

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/24/3524.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[0, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/82/4882.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[1, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/83/4883.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[2, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/84/4884.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[3, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/86/1420586.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[4, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/87/1420587.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[5, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/88/1420588.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[6, :, :] = npy[:, 0:1400]

file = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/npy/49/1420849.npy'
npy = np.load(file, allow_pickle=True)
npy_bat[7, :, :] = npy[:, 0:1400]

npy_bat = npy_bat.astype(np.float32)

In [103]:
npy = npy_bat
npy = torch.from_numpy(npy)

npy = npy.to(device)
print(npy.shape)
#Sigmoid

torch.Size([8, 96, 1400])


In [148]:
num = 10
sum = np.zeros((batch_size, 56))
for i in range(num):
    result = model(npy)
    result = result.cpu()
    result = result.detach().numpy()
    sum += result
mean = sum / num
np.max(mean)

0.29130616039037704

In [149]:
result_tag = []
for k in range(batch_size):
    sigmoid = mean[k]
    tag = tag_list[sigmoid>=0.5]
    
    if tag.size == 0:
        result_tag.append('nothing')
    else:
        result_tag.append(tag)
        
result_tag

['nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing']

In [152]:
print(mean[0])
print(mean[1])

[0.0093794  0.00743994 0.03490946 0.01870481 0.00081316 0.01465558
 0.00178963 0.00236941 0.01154559 0.01672218 0.02432997 0.03787954
 0.04485313 0.01761038 0.0036377  0.00474314 0.04624933 0.01481607
 0.06109475 0.0135147  0.00284035 0.02915319 0.00608341 0.0033884
 0.00378338 0.00512939 0.02380432 0.00231193 0.00046495 0.00592841
 0.02556837 0.0083008  0.00796282 0.00328631 0.03121127 0.01901062
 0.00867739 0.00308672 0.00945039 0.01400737 0.00278592 0.03751105
 0.00927484 0.01095989 0.00676296 0.00662445 0.00648093 0.00567205
 0.03477603 0.04457954 0.01907557 0.00742553 0.00146769 0.00553099
 0.01445034 0.03770784]
[2.24105367e-03 1.06180753e-02 1.75450447e-02 1.71330714e-02
 2.57286039e-02 5.82474869e-02 6.82702407e-03 2.93864482e-02
 2.69232377e-03 7.39911441e-03 4.86852821e-03 3.88153598e-02
 4.41818385e-03 2.67014323e-02 4.33326846e-02 7.95193511e-03
 1.10229938e-01 1.06410878e-01 7.94414783e-03 2.00314694e-02
 3.55215687e-04 8.40807192e-02 1.91088029e-03 8.03658362e-04
 4.92796

In [151]:
prd = np.load('prd.npy')
for i in range(4):
    print(prd[3])

[0.00944398 0.02797113 0.06501906 0.07922236 0.02496681 0.04341706
 0.02015148 0.06546817 0.03841611 0.02202723 0.05853711 0.05372874
 0.01289797 0.04313523 0.02184348 0.01814949 0.07711811 0.06561141
 0.07939976 0.0251214  0.00704786 0.05450741 0.03153061 0.00977086
 0.03051976 0.01033142 0.13386284 0.00530622 0.0209472  0.02488785
 0.09072892 0.08634143 0.03775924 0.02971969 0.10563992 0.07734646
 0.02097185 0.01655667 0.01161706 0.04926325 0.00900364 0.08217905
 0.0167083  0.06307996 0.05143759 0.00553336 0.02650961 0.03221552
 0.02676188 0.02559623 0.02043775 0.01314108 0.00403166 0.01884381
 0.02101695 0.0639491 ]
[0.00944398 0.02797113 0.06501906 0.07922236 0.02496681 0.04341706
 0.02015148 0.06546817 0.03841611 0.02202723 0.05853711 0.05372874
 0.01289797 0.04313523 0.02184348 0.01814949 0.07711811 0.06561141
 0.07939976 0.0251214  0.00704786 0.05450741 0.03153061 0.00977086
 0.03051976 0.01033142 0.13386284 0.00530622 0.0209472  0.02488785
 0.09072892 0.08634143 0.03775924 0.02

In [155]:
np.shape(prd)

(4231, 56)

In [154]:
count = prd>0.5
np.count_nonzero(count)

153